In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# import math
import numpy as np
from typing import Any, Optional, List
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import torchvision

In [ ]:
torch.backends.cudnn.version()

In [ ]:
torch.backends.cudnn.benchmark = True

In [ ]:
# import numpy as np

In [ ]:
import torch.autograd.profiler as profiler

In [ ]:
import pytorch_lightning as pl

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (18,12) # set default size of plots

In [ ]:
import stylegan, data_module

In [ ]:
gan = stylegan.GanTask(gamma=28, ppl_reg_every=4, penalize_d_every=16, resolution=512)

In [ ]:
gan = gan.load_from_checkpoint(checkpoint_path="./lightning_logs/version_30/checkpoints/epoch=3-step=23157.ckpt", strict=False)

In [ ]:
# dick = torch.load('new_synthesis_net.pt')
# gan.synthesis_net.load_state_dict(dick, strict=True)
# dick = torch.load('mapping_network.pt')
# gan.mapping_net.load_state_dict(dick, strict=True)


In [ ]:
dm = data_module.DataModule(data_dir='/mnt/data/datasets/caricatures/20200726_multimodal_celeba_images',
                                batch_size=16, 
                                balanced_sampling=True,resolution=512)

In [ ]:
# dm = data_module.DataModule(data_dir='./dataset',
#                                 batch_size=16, 
#                                 balanced_sampling=True,resolution=512)

In [ ]:
# dm.setup()

In [ ]:
# dl = dm.train_dataloader()

In [ ]:
# for batch in dl:
#     break

In [ ]:
# labels = F.one_hot(batch[1].long(), 2).float().cuda() #TODO: remove


In [ ]:
# gan = gan.cuda()

In [ ]:
# batch[0] = batch[0].cuda()

In [ ]:
# batch[1] = batch[1].cuda()

In [ ]:
# with profiler.profile(record_shapes=True, profile_memory=True, use_cuda=True, with_stack=True) as prof:
#     with profiler.record_function("model_inference"):
#         gan.training_step(batch, 0, 0)

In [ ]:
# with profiler.profile(record_shapes=True, profile_memory=True, use_cuda=True, with_stack=True) as prof:
#     with profiler.record_function("model_inference"):
#         gan(4, labels)

In [ ]:
# kav = prof.key_averages()

In [ ]:
# kav.total_average()

In [ ]:
# a = torch.randn(1000000, 3)
# b = torch.randn(1000000, 3)

# with torch.autograd.profiler.profile(use_cuda=True) as prof:
#     torch.mm(a.T, b)
# print(prof.key_averages().table(sort_by="self_cpu_time_total"))
# # Self CPU time total: 12.777ms
# # CUDA time total: 12.951ms

# a, b = a.cuda(), b.cuda()
# with torch.autograd.profiler.profile(use_cuda=True) as prof:
#     torch.mm(a.T, b)
# print(prof.key_averages().table(sort_by="self_cpu_time_total"))

In [ ]:
class Hooks(pl.Callback):
    num = 0
    one_hot = F.one_hot(
        torch.cat(
            (torch.zeros((3,)).long(),
            torch.ones((3,)).long())
        ), 2).float()

    def on_epoch_end(self, trainer, model: stylegan.GanTask):
        with torch.no_grad():
            fake_images, fake_latents = model.forward(6, self.one_hot.cuda())
            tensorboard = trainer.logger.experiment
            grid = torchvision.utils.make_grid(fake_images.cpu(), normalize=True, range=(-1, 1))
            tensorboard.add_image('images', grid, self.num)
            self.num += 1     

In [ ]:
trainer = pl.Trainer(
    benchmark=True,
    automatic_optimization=False, gpus=1, 
    accumulate_grad_batches=1, 
    precision=32,
    callbacks=[Hooks()],
    default_root_dir = '/home/duke/store/artefacts/limner/stylegan2_caricatures',
#   amp_backend='native', gradient_clip_val=10,
#   amp_level='01',
#   terminate_on_nan=True
)

In [ ]:
trainer.fit(gan, dm)

In [ ]:
gan = gan.cpu()

In [ ]:
trainer.save_checkpoint("model_final_3.ckpt")
# new_model = MyModel.load_from_checkpoint(checkpoint_path="example.ckpt")


In [ ]:
fake_images.requires_grad

In [ ]:
# with torch.no_grad():
gan = gan.cuda()
fake_images, fake_latents = gan.forward(8, F.one_hot(torch.ones((8,)).long(), 2).float().cuda(), mix_prob=1)
# fake_latents.requires_grad_(True)
# fake_images.requires_grad_(True)
path_loss, path_lengths = gan.ppl_regularization(fake_images.cuda(), fake_latents.cuda())

In [ ]:
path_loss

In [ ]:
path_lengths

In [ ]:
# path_loss, path_lengths = gan.ppl_regularization(fake_images, fake_latents)

In [ ]:
# gan.requires_grad_(True)

In [ ]:
# fake_images, fake_latents = gan.forward(8, F.one_hot(torch.zeros((8,)).long(), 2).float(), mix_prob=0)
# path_loss, path_lengths = gan.ppl_regularization(fake_images, fake_latents)

In [ ]:
# path_lengths

In [ ]:
with torch.no_grad():
    fake_images, fake_latents = gan(8, F.one_hot(torch.ones((8,)).long(), 2).float())

In [ ]:
fake_latents = 

In [ ]:
path_lengths

In [ ]:
def make_image(tensor):
    return (
        tensor.detach()
        .clamp_(min=-1, max=1)
        .add(1)
        .div_(2)
        .mul(255)
        .type(torch.uint8)
        .permute(1, 2, 0)
        .to("cpu")
        .numpy())

In [ ]:
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [ ]:
noise = gan.synthesis_net.make_noise(1)

In [ ]:
latents_all = []

with torch.no_grad():
    n = 2048
    for i in range(2):
        z = torch.randn([n, 512]).float()
        l = F.one_hot(torch.ones((n,)).long() * i, 2).float()
        latents_princeses = gan.mapping_net.forward(z, l)
        latent_mean = torch.mean(latents_princeses, 0)
        latent_plus_mean = latent_mean[None, None, :].repeat([1, 16, 1])
        latents_all.append(latents_princeses)

In [ ]:
fake_latents = latents_princeses[:, None, :].repeat([1, 16, 1])

In [ ]:
ilya1 = np.load('./ilya1.npy')
ilya2 = np.load('./ilya2.npy')
ilya3 = np.load('./ilya3.npy')
ilya4 = np.load('./ilya4.npy')

In [ ]:
fake_latents[0] = torch.from_numpy(ilya1)
fake_latents[1] = torch.from_numpy(ilya2)
fake_latents[2] = torch.from_numpy(ilya3)
fake_latents[3] = torch.from_numpy(ilya4)

In [ ]:
@interact
def play_with_mix(i=(0,len(fake_latents),1), j=(0,len(fake_latents),1), c=(0,1,0.05), nc=(0,1,0.1), truncation=(0.0,1.0,0.1)):
    with torch.no_grad():
    #     print(path_lengths[j].item())
        n = torch.randn_like(fake_latents[i][None, :, :]) * 0.5
        latend = (c * fake_latents[i][None, :, :] + (1 -c) * fake_latents[j][None, :, :])

        latent_plus = latend + n * nc
        latent_plus = latent_plus_mean.lerp(latent_plus, truncation)

        old_img = make_image(gan.synthesis_net.forward(latent_plus, noise)[0])
        ema_img = make_image(gan.synthesis_net_ema.forward(latent_plus, noise)[0])

        fig = plt.figure()
        ax1 = fig.add_subplot(1,2,1)
        ax1.imshow(old_img)
        ax1.grid(False)
        ax2 = fig.add_subplot(1,2,2)
        ax2.imshow(ema_img)
        ax2.grid(False)
        plt.show()

# PCA 

In [ ]:
from types import SimpleNamespace
import fbpca
import itertools

In [ ]:
# Facebook's PCA
# Good default choice: very fast and accurate.
# Very high sample counts won't fit into RAM,
# in which case IncrementalPCA must be used.
class FacebookPCAEstimator():
    def __init__(self, n_components):
        self.n_components = n_components
        self.transformer = SimpleNamespace()
        self.batch_support = False
        self.n_iter = 2
        self.l = 2*self.n_components

    def get_param_str(self):
        return "fbpca_c{}_it{}_l{}".format(self.n_components, self.n_iter, self.l)

    def fit(self, X):
        U, s, Va = fbpca.pca(X, k=self.n_components, n_iter=self.n_iter, raw=True, l=self.l)
        self.transformer.components_ = Va
        
        # Save variance for later
        self.total_var = X.var(axis=0).sum()

        # Compute projected standard deviations
        self.stdev = np.dot(self.transformer.components_, X.T).std(axis=1)

        # Sort components based on explained variance
        idx = np.argsort(self.stdev)[::-1]
        self.stdev = self.stdev[idx]
        self.transformer.components_[:] = self.transformer.components_[idx]

        # Check orthogonality
        dotps = [np.dot(*self.transformer.components_[[i, j]])
            for (i, j) in itertools.combinations(range(self.n_components), 2)]
        if not np.allclose(dotps, 0, atol=1e-4):
            print('FBPCA components not orghogonal, max dot', np.abs(dotps).max())

        self.transformer.mean_ = X.mean(axis=0, keepdims=True)
        
    def get_components(self):
        var_ratio = self.stdev**2 / self.total_var
        return self.transformer.components_, self.stdev, var_ratio

In [ ]:
pile_of_latents = []
n = 2048

with torch.no_grad():
    for label in range(2):
        for _ in range(64):
            z = torch.randn([n, 512]).float()
            l = F.one_hot(torch.ones((n,)).long() * label, 2).float()
            latents_princeses = gan.mapping_net.forward(z, l)
            pile_of_latents.append(latents_princeses.cpu().detach().numpy())


In [ ]:
latentz =  np.concatenate(pile_of_latents)

In [ ]:
estimator = FacebookPCAEstimator(n_components=512)

In [ ]:
estimator.fit(F.leaky_relu(torch.tensor(latentz), negative_slope=5.0))

In [ ]:
X_comp, X_stdev, X_var_ratio = estimator.get_components()

In [ ]:
X_comp_norm = X_comp /  np.linalg.norm(X_comp, axis=-1, keepdims=True)

In [ ]:
X_comp_norm = F.leaky_relu(torch.tensor(X_comp_norm), negative_slope=0.2)

In [ ]:
# np.save('pca_vectors.npy', X_comp_norm)

In [ ]:
computed_latentz =  fake_latents

In [ ]:
latent_plus_mean.shape

In [ ]:
X_comp_norm.shape

In [ ]:
@interact
def mix(index = (0, 511, 1),
        shift = (-40,40,0.5),
        index_2 = (0, 511, 1),
        shift_2 = (-40,40,0.5),
        index_3 = (0, 511, 1),
        shift_3 = (-40,40,0.5),
        latent_index = (0, len(computed_latentz) - 1, 1),
        artefacts_shift = (0,30,0.5),
        cari_shift = (0,12,0.5),
        truncation=(0.0,1.0,0.1),
       save = False,
       write_video = False):

    latent = computed_latentz[latent_index] 
    latent = latent_plus_mean[0].lerp(latent, truncation)

    
    direction = X_comp_norm[int(index)] 
    direction_2 = X_comp_norm[int(index_2)] 
    direction_3 = X_comp_norm[int(index_3)] 
    
    shifted_latent = latent + shift * direction + cari_shift * X_comp_norm[int(0)] - artefacts_shift * X_comp_norm[int(11)]  
    shifted_latent_minus = latent - shift * direction + cari_shift * X_comp_norm[int(0)] - artefacts_shift * X_comp_norm[int(11)]  
    
    shifted_latent += shift_2 * direction_2
    shifted_latent_minus -= shift_2 * direction_2
    
    shifted_latent += shift_3 * direction_3
    shifted_latent_minus -= shift_3 * direction_3

    
    latents_ = np.stack([shifted_latent_minus, latent, shifted_latent])
    
    with torch.no_grad():
        noise = gan.synthesis_net.make_noise(3)
        #print(latents_.shape)
        rezults = gan.synthesis_net_ema.forward(torch.from_numpy(latents_).float(), noise)

    out_minus = make_image(rezults[0])
    src = make_image(rezults[1])
    img = make_image(rezults[2])
    
#     img = np.clip(img, 0, 1)
#     out_minus = np.clip(out_minus, 0, 1)
#     src = np.clip(src, 0, 1)
    
    if save:
        np.save(f'latent_index_{latent_index}_shift_{shift}_{shift_2}_{shift_3}_index_{index}_{index_2}_{index_3}_artefacts_shift_{artefacts_shift}.npy', shifted_latent)
        np.save(f'latent_index_{latent_index}_shift_-{shift}_{shift_2}_{shift_3}_index_{index}_{index_2}_{index_3}_artefacts_shift_{artefacts_shift}.npy', shifted_latent_minus)
        plt.imsave(f'latent_index_{latent_index}_shift_{shift}_{shift_2}_{shift_3}_index_{index}_{index_2}_{index_3}_artefacts_shift_{artefacts_shift}.png', img)

#     if write_video:
#         fid = f'{latent_index}_-_shift_{shift}_{shift_2}_{shift_3}_index_{index}_{index_2}_{index_3}_{artefacts_shift}_{cari_shift}'
#         make_video(synthesis_net, latent, shifted_latent_minus, f'latents/512/{fid}')
#         plt.imsave(f'latents/512/{fid}/final.png', out_minus)

#         fid = f'{latent_index}__shift_{shift}_{shift_2}_{shift_3}_index_{index}_{index_2}_{index_3}_{artefacts_shift}_{cari_shift}'
#         make_video(synthesis_net, latent, shifted_latent, f'latents/512/{fid}')
#         plt.imsave(f'latents/512/{fid}/final.png', img)

        

    fig = plt.figure()
    ax1 = fig.add_subplot(1,3,1)
    ax1.imshow(out_minus)
    ax1.grid(False)
    ax2 = fig.add_subplot(1,3,2)
    ax2.imshow(src)
    ax2.grid(False)
    ax3 = fig.add_subplot(1,3,3)
    ax3.imshow(img)
    ax3.grid(False)

# Generate files

In [ ]:
for image_index in range(2048):
    truncation = 0.6
    latent = computed_latentz[image_index] 
    latent = latent_plus_mean[0].lerp(latent, truncation)

    index = 16
    index_2 = 351
    index_3 = 15

    shift = 1
    shift_2 = -4
    shift_3 = -3


    direction = X_comp_norm[int(index)] 
    direction_2 = X_comp_norm[int(index_2)] 
    direction_3 = X_comp_norm[int(index_3)] 

    shifted_latent = latent
    # shifted_latent += shift * direction
    # shifted_latent += shift_2 * direction_2
    # shifted_latent += shift_3 * direction_3

    latents_ = np.stack([shifted_latent])

    with torch.no_grad():
        noise = gan.synthesis_net.make_noise(1)
        #print(latents_.shape)
        rezults = gan.synthesis_net_ema.forward(torch.from_numpy(latents_).float(), noise)

    img = make_image(rezults[0])

    plt.imsave(f'/mnt/sdd3/Limner/GeneratedCartoon/{3072 + image_index}.png', img)

In [ ]:
fake_latents.shape

In [ ]:
# flatents = fake_latents[:,0]

In [ ]:
with torch.no_grad():
    fake_images, fake_latents = gan.forward(8, F.one_hot(torch.zeros((8,)).long(), 2).float(), mix_prob=1)

In [ ]:
flatents = torch.flatten(fake_latents,start_dim=1)

In [ ]:
# flatents_photo = torch.flatten(fake_latents_photo,start_dim=1)


In [ ]:
torch.cdist(flatents, flatents)[0] 

In [ ]:
with torch.no_grad():
    img = gan.synthesis_net(fake_latents)
    lblz = F.one_hot(torch.zeros((8,)).long(), 2).float()
    _, activations = gan.critic_net.forward(img, lblz, return_activations=True)
    projection = gan.auxiliary_projector.forward(activations[-1])

In [ ]:
torch.cdist(projection, projection)[0] 

In [ ]:
F.mse_loss(torch.cdist(flatents, flatents)[0] , torch.cdist(projection, projection)[0] )

In [ ]:
import scipy

In [ ]:
from scipy import spatial

In [ ]:
bath_size = 32
with torch.no_grad():
    latents = []
    for _ in range(128):
        z = torch.randn((bath_size, 512))
        l =  F.one_hot(torch.zeros((bath_size, )).long(), 2).float()
        w = gan.mapping_net.forward(z, l).detach().numpy()
        latents.append(w)

In [ ]:
bath_size = 32
with torch.no_grad():
    latents_photo = []
    for _ in range(128):
        z = torch.randn((bath_size, 512))
        l =  F.one_hot(torch.ones((bath_size, )).long(), 2).float()
        w = gan.mapping_net.forward(z, l).detach().numpy()
        latents_photo.append(w)

In [ ]:
def unfuck(x):
    return np.maximum(0, x) + 5.0 * np.minimum(0, x)

In [ ]:
latents_photo =  np.concatenate(latents_photo, axis=0)

In [ ]:
latents = np.concatenate(latents, axis=0)

In [ ]:
np.save('latents.npy', unfuck(latents))
np.save('latents_photo.npy', unfuck(latents_photo))

In [ ]:
for i in range(32):
    _ = plt.hist(unfuck(latents[:, i]), bins=100)
    _ = plt.hist(unfuck(latents_photo[:, i]), bins=100, alpha=0.5)
    plt.show()

In [ ]:
d = spatial.distance.cdist(latents, latents)

In [ ]:
d_id = spatial.distance.cdist(latents, latents_photo)

In [ ]:
d_p = spatial.distance.cdist(latents_photo, latents_photo)

In [ ]:
d_flat = d.ravel()[d.ravel() > 0]

In [ ]:
d_id_flat = d_id.ravel()[d_id.ravel() > 0]

In [ ]:
d_p = d_p.ravel()[d_p.ravel() > 0]

In [ ]:
plt.hist(d_flat, bins=100)
plt.hist(d_id_flat, bins=100, alpha=0.5)
plt.hist(d_p, bins=100, alpha=0.5, label='photos')

In [ ]:
r = torch.randperm(8)
r

In [ ]:
torch.pairwise_distance(a1_flat, a1_flat[r])

In [ ]:
with torch.no_grad():
    gan.critic_net(a[0], F.one_hot(torch.randint(0, 2, (4,)), 2).float())

In [ ]:
dl = dm.train_dataloader()

In [ ]:
for imz, lbz in dl:
    break

In [ ]:
lbz[1]

In [ ]:
a_cpu = make_image(imz[3])
plt.imshow(a_cpu)

37/40875 [03:16<60:23:08, 5.32s/it, loss=nan, v_num=31, path_lengths=0.379, generator_loss=2.03, critic_loss=1.42

In [ ]:
skorez = gan.critic_net(imz, F.one_hot(lbz, 2))

In [ ]:
jopa = torch.mean(skorez)

In [ ]:
jopa.backward()

In [ ]:
skorez.grad

In [ ]:
gan.

In [ ]:
a = gan(8, F.one_hot(torch.randint(0, 2, (8,)), 2).float())

In [ ]:
jopel = torch.mean(a[0])

In [ ]:
jopel.backward()

In [ ]:
jopel.grad